# Deep Reinforcement Learning: Algorithm Comparison

This notebook demonstrates and compares four actor-critic algorithms for continuous control:
- **DDPG** - Deep Deterministic Policy Gradient (original)
- **OurDDPG** - Re-tuned DDPG with 256-256 architecture
- **TD3** - Twin Delayed Deep Deterministic Policy Gradient
- **QRTD3** - Quantile Regression TD3

All implementations are imported from the Python files in this repository.


## Configuration

Set the environment and seed here. You can also choose which algorithm to run.


In [ ]:
# Configuration - Set these at the beginning
ENV_NAME = "Reacher-v5"  # Options: Reacher-v5, Ant-v5, HalfCheetah-v5, Hopper-v5, Walker2d-v5, etc.
SEED = 0
POLICY = "TD3"  # Options: "TD3", "DDPG", "OurDDPG", "QRTD3"

# Training hyperparameters
MAX_TIMESTEPS = int(1e6)  # 1M timesteps
START_TIMESTEPS = int(25e3)  # Initial random exploration
EVAL_FREQ = int(5e3)  # Evaluation frequency
BATCH_SIZE = 256
DISCOUNT = 0.99
TAU = 0.005
POLICY_NOISE = 0.2
NOISE_CLIP = 0.5
POLICY_FREQ = 2
EXPL_NOISE = 0.1

print(f"Configuration:")
print(f"  Environment: {ENV_NAME}")
print(f"  Seed: {SEED}")
print(f"  Policy: {POLICY}")
print(f"  Max timesteps: {MAX_TIMESTEPS:,}")


Configuration:
  Environment: Reacher-v5
  Seed: 0
  Policy: TD3
  Max timesteps: 1,000,000


## Imports

Import the necessary libraries and algorithm implementations from the Python files.


In [ ]:
import numpy as np
import torch
import gymnasium as gym
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import os

# Import algorithm implementations
import utils
import TD3
import DDPG
import OurDDPG
import QR_TD3

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")


Using device: cpu
PyTorch version: 2.9.1


## Algorithm Overview

### DDPG (Deep Deterministic Policy Gradient)
- **Paper**: [Continuous control with deep reinforcement learning](https://arxiv.org/abs/1509.02971) (Lillicrap et al., 2015)
- Single critic network (400-300 architecture)
- Actor network (400-300 architecture)
- Updates policy every step
- Learning rate: 1e-4 (actor), weight_decay=1e-2 (critic)
- Tau: 0.001

### OurDDPG (Re-tuned DDPG)
- Same as DDPG but with 256-256 architecture
- Learning rate: 3e-4
- Tau: 0.005
- Batch size: 256

### TD3 (Twin Delayed Deep Deterministic Policy Gradient)
- **Paper**: [Addressing Function Approximation Error in Actor-Critic Methods](https://arxiv.org/abs/1802.09477) (Fujimoto et al., 2018)
- **Key improvements over DDPG**:
  1. **Clipped Double Q-Learning**: Uses two critic networks, takes minimum to reduce overestimation
  2. **Delayed Policy Updates**: Updates policy every 2 steps (less frequent than critics)
  3. **Target Policy Smoothing**: Adds clipped noise to target actions
- Twin critics (256-256 architecture each)
- Actor network (256-256 architecture)
- Learning rate: 3e-4
- Tau: 0.005

### QRTD3 (Quantile Regression TD3)
- Extension of TD3 using quantile regression for the critic
- Uses quantile loss instead of MSE loss
- Can better capture uncertainty in value estimates
- Same architecture as TD3 but with quantile-based critics


## Evaluation Function

This function evaluates the policy over multiple episodes without exploration noise.


In [ ]:
def eval_policy(policy, env_name, seed, eval_episodes=10):
    """Evaluate the policy over multiple episodes."""
    eval_env = gym.make(env_name)
    avg_reward = 0.
    for _ in tqdm(range(eval_episodes), desc="Evaluating", leave=False):
        state = eval_env.reset(seed=seed + 100)[0]
        done = False
        while not done:
            action = policy.select_action(np.array(state))
            state, reward, terminated, truncated, _ = eval_env.step(action)
            done = terminated or truncated
            avg_reward += reward
    
    avg_reward /= eval_episodes
    print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f}")
    return avg_reward


## Training Setup

Initialize the environment and policy based on the configuration.


In [ ]:
# Create environment
env = gym.make(ENV_NAME)

# Set seeds
env.action_space.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

# Get environment dimensions
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = float(env.action_space.high[0])

print(f"State dimension: {state_dim}")
print(f"Action dimension: {action_dim}")
print(f"Max action: {max_action}")

# Initialize policy
kwargs = {
    "state_dim": state_dim,
    "action_dim": action_dim,
    "max_action": max_action,
    "discount": DISCOUNT,
    "tau": TAU,
}

if POLICY == "TD3":
    kwargs["policy_noise"] = POLICY_NOISE * max_action
    kwargs["noise_clip"] = NOISE_CLIP * max_action
    kwargs["policy_freq"] = POLICY_FREQ
    policy = TD3.TD3(**kwargs)
elif POLICY == "QRTD3":
    kwargs["policy_noise"] = POLICY_NOISE * max_action
    kwargs["noise_clip"] = NOISE_CLIP * max_action
    kwargs["policy_freq"] = POLICY_FREQ
    policy = QR_TD3.QRTD3(**kwargs)
elif POLICY == "OurDDPG":
    policy = OurDDPG.DDPG(**kwargs)
elif POLICY == "DDPG":
    policy = DDPG.DDPG(**kwargs)
else:
    raise ValueError(f"Unknown policy: {POLICY}")

print(f"\nPolicy {POLICY} initialized successfully!")

# Initialize replay buffer
replay_buffer = utils.ReplayBuffer(state_dim, action_dim)
print(f"Replay buffer initialized")


State dimension: 10
Action dimension: 2
Max action: 1.0

Policy TD3 initialized successfully!
Replay buffer initialized


## Training Loop

The training loop follows the standard off-policy actor-critic procedure:
1. Collect experience by interacting with the environment
2. Store transitions in the replay buffer
3. Sample batches and update the policy
4. Periodically evaluate the policy without exploration noise


In [ ]:
# Evaluate untrained policy
print("Evaluating untrained policy...")
evaluations = [eval_policy(policy, ENV_NAME, SEED)]

# Initialize training variables
state = env.reset(seed=SEED + 100)[0]
done = False
episode_reward = 0
episode_timesteps = 0
episode_num = 0

start_time = time.time()

print(f"\nStarting training for {MAX_TIMESTEPS:,} timesteps...")
print(f"Random exploration for first {START_TIMESTEPS:,} steps")
print(f"Evaluation frequency: every {EVAL_FREQ:,} steps\n")

# Main training loop
for t in tqdm(range(int(MAX_TIMESTEPS)), desc="Training"):
    episode_timesteps += 1
    
    # Select action randomly or according to policy with exploration noise
    if t < START_TIMESTEPS:
        action = env.action_space.sample()
    else:
        action = (
            policy.select_action(np.array(state))
            + np.random.normal(0, max_action * EXPL_NOISE, size=action_dim)
        ).clip(-max_action, max_action)
    
    # Perform action in environment
    next_state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated
    done_bool = float(done) if episode_timesteps < env._max_episode_steps else 0
    
    # Store transition in replay buffer
    replay_buffer.add(state, action, next_state, reward, done_bool)
    
    state = next_state
    episode_reward += reward
    
    # Train agent after collecting sufficient data
    if t >= START_TIMESTEPS:
        policy.train(replay_buffer, BATCH_SIZE)
    
    if done:
        # Reset environment
        state = env.reset(seed=SEED + 100)[0]
        done = False
        episode_reward = 0
        episode_timesteps = 0
        episode_num += 1
    
    # Evaluate episode
    if (t + 1) % EVAL_FREQ == 0:
        eval_reward = eval_policy(policy, ENV_NAME, SEED)
        evaluations.append(eval_reward)

end_time = time.time()
duration = end_time - start_time
hours = int(duration // 3600)
minutes = int((duration % 3600) // 60)
seconds = int(duration % 60)

print("\n" + "="*70)
print("Training completed!")
print(f"Training time: {hours:02d}:{minutes:02d}:{seconds:02d}")
print(f"Final evaluation reward: {evaluations[-1]:.3f}")
print("="*70)


Evaluating untrained policy...


Evaluation over 10 episodes: -5.469

Starting training for 1,000,000 timesteps...
Random exploration for first 25,000 steps
Evaluation frequency: every 5,000 steps



Training:   1%|          | 6802/1000000 [00:00<01:42, 9720.69it/s] 

Evaluation over 10 episodes: -5.469


Training:   1%|          | 11350/1000000 [00:01<01:43, 9563.88it/s]

...

Training:  41%|████      | 410012/1000000 [50:38<1:51:33, 88.14it/s] 

Evaluation over 10 episodes: -0.928


Training:  42%|████▏     | 415018/1000000 [51:31<1:55:58, 84.07it/s] 

Evaluation over 10 episodes: -0.848


Training:  42%|████▏     | 419073/1000000 [52:16<5:32:34, 29.11it/s] 

## Results Visualization

Plot the learning curve showing how the average reward improves over time.


In [ ]:
# Plot learning curve
timesteps = np.array([0 if i == 0 else i * EVAL_FREQ for i in range(len(evaluations))])

plt.figure(figsize=(12, 6))
plt.plot(timesteps, evaluations, marker='o', linewidth=2, markersize=4, label=POLICY)
plt.xlabel('Timestep', fontsize=14)
plt.ylabel('Average Reward', fontsize=14)
plt.title(f'{POLICY} Learning Curve on {ENV_NAME} (Seed {SEED})', fontsize=16, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nPerformance Summary:")
print(f"  Initial reward: {evaluations[0]:.3f}")
print(f"  Final reward: {evaluations[-1]:.3f}")
print(f"  Improvement: {evaluations[-1] - evaluations[0]:.3f}")
print(f"  Total evaluations: {len(evaluations)}")


## Summary

This notebook demonstrated how to:
1. Import algorithm implementations from Python files
2. Configure environment and hyperparameters
3. Train an agent using the off-policy actor-critic framework
4. Visualize learning curves

To compare different algorithms, simply change the `POLICY` variable at the top and re-run the training cells. All algorithms share the same interface (`select_action`, `train`), making it easy to compare them.

### Key Differences Between Algorithms

| Algorithm | Critic Networks | Policy Updates | Special Features |
|-----------|----------------|----------------|------------------|
| DDPG | Single (400-300) | Every step | - |
| OurDDPG | Single (256-256) | Every step | Re-tuned hyperparameters |
| TD3 | Twin (256-256) | Every 2 steps | Clipped Double Q, Target Smoothing |
| QRTD3 | Twin Quantile (256-256) | Every 2 steps | Quantile Regression |

For more details, see the individual algorithm files: `DDPG.py`, `OurDDPG.py`, `TD3.py`, `QR_TD3.py`
